<a href="https://colab.research.google.com/github/arpit-devop/summeranalyticsIITG/blob/main/analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install pathway bokeh geopy --quiet

import pathway as pw
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from geopy.distance import geodesic
import asyncio
output_notebook()


In [51]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Capacity: float
    Latitude: float
    Longitude: float
    Occupancy: float
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: float
    IsSpecialDay: float
    LastUpdatedDate: str
    LastUpdatedTime: str



In [52]:


csv_path = '/content/dataset.csv'
stream = pw.demo.replay_csv(
    path=csv_path,
    schema=ParkingSchema,
    input_rate=2
)


In [53]:
def vehicle_type_weight(vtype):
    return {'car': 1.0, 'bike': 0.7, 'truck': 1.5, 'cycle': 0.5}.get(str(vtype).lower(), 1.0)

def traffic_level_num(traffic):
    return {'low': 0, 'average': 0.5, 'high': 1}.get(str(traffic).lower(), 0.5)

def demand_function(row):
    alpha, beta, gamma, delta, epsilon = 1, 0.5, 0.3, 2, 1
    return (
        alpha * (row['Occupancy'] / row['Capacity'] if row['Capacity'] > 0 else 0) +
        beta * row['QueueLength'] -
        gamma * traffic_level_num(row['TrafficConditionNearby']) +
        delta * row['IsSpecialDay'] +
        epsilon * vehicle_type_weight(row['VehicleType'])
    )


In [54]:
@pw.udf
def compute_price(
    Occupancy, Capacity, QueueLength, TrafficConditionNearby, IsSpecialDay, VehicleType
):

    BASE_PRICE = 10.0
    lambda_ = 0.8

    demand = demand_function({
        'Occupancy': Occupancy,
        'Capacity': Capacity,
        'QueueLength': QueueLength,
        'TrafficConditionNearby': TrafficConditionNearby,
        'IsSpecialDay': IsSpecialDay,
        'VehicleType': VehicleType
    })

    norm_demand = min(max(demand / 10, 0), 1)
    price = BASE_PRICE * (1 + lambda_ * norm_demand)
    price = np.clip(price, 0.5 * BASE_PRICE, 2 * BASE_PRICE)
    return float(price)


stream = stream.with_columns(
    price=pw.apply(
        compute_price,
        stream.Occupancy,
        stream.Capacity,
        stream.QueueLength,
        stream.TrafficConditionNearby
      ,
        stream.IsSpecialDay,
        stream.VehicleType
    )
)

In [55]:

print(stream.schema)

id          | SystemCodeNumber | Capacity | Latitude | Longitude | Occupancy | VehicleType | TrafficConditionNearby | QueueLength | IsSpecialDay | LastUpdatedDate | LastUpdatedTime | price
ANY_POINTER | STR              | FLOAT    | FLOAT    | FLOAT     | FLOAT     | STR         | STR                    | FLOAT       | FLOAT        | STR             | STR             | ANY  


In [56]:
import pathway as pw
import pandas as pd


class ParkingData(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: bool
    LastUpdatedDate: str
    LastUpdatedTime: str
    price: float


stream = pw.io.csv.read("input.csv", schema=ParkingData)


selected_stream = stream.select(
    SystemCodeNumber=stream.SystemCodeNumber,
    Capacity=stream.Capacity,
    Latitude=stream.Latitude,
    Longitude=stream.Longitude,
    Occupancy=stream.Occupancy,
    VehicleType=stream.VehicleType,
    TrafficConditionNearby=stream.TrafficConditionNearby,
    QueueLength=stream.QueueLength,
    IsSpecialDay=stream.IsSpecialDay,
    LastUpdatedDate=stream.LastUpdatedDate,
    LastUpdatedTime=stream.LastUpdatedTime,
    Price=stream.price
)



In [57]:

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook


source = ColumnDataSource(data={'time': [], 'price': [], 'parking_lot_id': []})

p = figure(x_axis_label="Time (relative)", y_axis_label="Price ($)",
           title="Real-time Parking Price Updates",
           height=300, width=800, tools="pan,wheel_zoom,box_zoom,reset,save")


p.line(x='time', y='price', source=source, legend_field='parking_lot_id', line_width=2)

p.legend.click_policy = "hide"


show(p, notebook_handle=True)

In [58]:
import pathway as pw

@pw.udf
def compute_price(
    occupancy: int,
    capacity: int,
    queue_length: int,
    traffic_level: str,
    is_special_day: int,
    vehicle_type: str,
) -> float:
    α = 1.0
    β = 0.5
    γ_map = {"low": 0.2, "average": 0.5, "high": 1.0}
    δ = 1.0
    ε_map = {"car": 0.3, "bike": 0.1, "truck": 0.5}

    γ = γ_map.get(traffic_level.lower(), 0.5)
    ε = ε_map.get(vehicle_type.lower(), 0.3)

    demand = α * (occupancy / capacity) + β * queue_length - γ + δ * is_special_day + ε
    demand = min(max(demand, 0), 2)

    base_price = 10
    λ = 0.5
    price = base_price * (1 + λ * demand)

    return round(price, 2)
